<a href="https://colab.research.google.com/github/rfprogra/tp-information-retrieval-with-llm-student-version/blob/main/Copie_de_1_Recherche_d'information_classique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1. - Recherche d'Information classique

Dans cette partie, nous allons mettre en oeuvre des principes et modèles classiques de Recherche d'Information. Le jeu de donnée est une collection de livres au format texte (.txt) de Henry Rider Haggard. Jetez un oeil à ces documents dans le dossier _data_.

En sortie de ce module, vous serez capable de :

- Construire un index inversé ;
- Effectuer des requêtes simples selon le modèle booléen :
- Calculer la pondération des termes selon la méthode TF-IDF ;
- Mettre en oeuvre le modèle vectoriel de recherche d'information et y appliquer des requêtes.

### Import des bibliothèques logicielles et configuration

Les lignes suivantes permettent d'instancier un objet la classe `IRSystem` représentant notre moteur de recherche et de charger les données en RAM.

In [2]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/rfprogra/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'tp-information-retrieval-with-llm-student-version'...
remote: Enumerating objects: 2253, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 2253 (delta 24), reused 22 (delta 12), pack-reused 2212
Receiving objects: 100% (2253/2253), 13.99 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/tp-information-retrieval-with-llm-student-version


#### Chargement des données

Les lignes ci-dessous permettent de charger les données qui sont un ensemble de 60 livres au format texte (.txt) d'[Henry Rider Haggard ](https://fr.wikipedia.org/wiki/Henry_Rider_Haggard).


In [3]:
from classic_ir.IRSystem import *

# !rm -rf ./data/RiderHaggard/stemmed
ir_system = IRSystem()
ir_system.read_data('./data/RiderHaggard') # chargement des données et prétraitement des documents (stemming).

Reading in documents...
Stemming Documents...
The Ivory Child 2841.txt
    Doc 1 of 60: The Ivory Child
Eric Brighteyes 2721.txt
    Doc 2 of 60: Eric Brighteyes
Red Eve 3094.txt
    Doc 3 of 60: Red Eve
Jess 5898.txt
    Doc 4 of 60: Jess
Stories by English Authors Africa (Selected by Scribners) 1980.txt
    Doc 5 of 60: Stories by English Authors Africa (Selected by Scribners)
Love Eternal 3709.txt
    Doc 6 of 60: Love Eternal
Child of Storm 1711.txt
    Doc 7 of 60: Child of Storm
Marie An Episode in The Life of the late Allan Quatermain 1690.txt
    Doc 8 of 60: Marie An Episode in The Life of the late Allan Quatermain
The Wizard 2893.txt
    Doc 9 of 60: The Wizard
Ayesha, the Return of She 5228.txt
    Doc 10 of 60: Ayesha, the Return of She
Lysbeth, a Tale of the Dutch 5754.txt
    Doc 11 of 60: Lysbeth, a Tale of the Dutch
The Wanderer's Necklace 3097.txt
    Doc 12 of 60: The Wanderer's Necklace
The Lady of Blossholme 3813.txt
    Doc 13 of 60: The Lady of Blossholme
Black He

### Exercice 1. - Construction de l'index inversé

Ce premier exercice a pour objectif de construire l'index inversé non positionnel. L'attribut `self.inverted_index` est un tableau associatif qui associe une liste d'entiers (docIDs) à un mot (word).

Documentation ici https://docs.python.org/3/library/collections.html#collections.defaultdict.

Exercice : modifier la fonction `index` pour calculer l'index inversé.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [4]:
# Exercice 1. Indexation

def index(self):
    """
    Construit l'index inversé et le stocke dans self.inverted_index.

    Dans le code ci-dessous, seul le dictionnaire des tokens est construit. Les postings lists sont vides.
    """
    print("Indexing...")
    self.tf = defaultdict(Counter) # tf est un dictionnaire qui à un document associe un Counter de mots.
    inverted_index = defaultdict(list) # inverted_index est un dictionnaire qui à un mot associe une liste de documents.

    i=0
    for doc in self.docs :
      for term in doc :
        if i not in inverted_index[term]:
          inverted_index[term].append(i);
      i=i+1

    self.inverted_index = inverted_index

# Ne pas modifier les lignes ci-dessous
IRSystem.index = index
ir_system.index()
run_tests(ir_system, part=0)

Indexing...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 2. - Recherche booléenne

Ce deuxième exercice a pour objectif d'implémenter la recherche booléenne. La requête `query` est une liste de mots _lemmatisés_ et l'algorithme doit rechercher les documents qui contiennent TOUS ces mots.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.


Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [5]:
# Exercice 2. Recherche booléenne
def boolean_retrieve(self, query):
    """
    A partir d'une requête sous la forme d'une liste de mots *lemmatisés*,
    retourne la liste des documents dans lesquels *tous* ces mots apparaissent (ie une requête AND).
    Retourne une liste vide si la requête ne retourne aucun document.

    Dans le code ci-dessous, tous les documents répondent.
    """
    docs = list()
    i=0
    for doc in self.docs:
      inside = True
      for lemm in query :
        if lemm not in doc :
          inside=False
      if inside == True :
        docs.append(i)
      i=i+1
    return docs

# Ne pas modifier les lignes ci-dessous
IRSystem.boolean_retrieve = boolean_retrieve
run_tests(ir_system, part=1)

===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 3. - Calcul des poids TF-IDF des termes dans les documents

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours. Utiliser le logarithme en base 10.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [6]:
# Exercice 3. calcul des scores tf-idf
def compute_tfidf(self):
    """
    Calcule les scores tf-idf pour tous les mots de tous les documents et les stocke dans self.tfidf.

    Dans le code ci-dessous, les scores tf-idf sont tous nuls.
    """
    print("Calculating tf-idf...")

    self.tfidf = defaultdict(Counter)
    N = len(self.docs)  # N = nombre de documents

    i=0
    tf=defaultdict(Counter)
    for doc in self.docs:
      for word in doc :
        tf[i][word] += 1
      i=i+1

    dft=defaultdict(Counter)
    for word in self.vocab:
        dft[word]=0
        for i in range(N):
          if  tf[i][word]!=0:
            dft[word]=dft[word]+1

    for word in self.vocab:
        for i in range(N):
          try:
              self.tfidf[i][word] = (1+math.log10( tf[i][word]))*math.log10(N/dft[word])
          except ValueError:
              self.tfidf[i][word] = 0.

# Ne pas modifier les lignes ci-dessous
IRSystem.compute_tfidf = compute_tfidf
ir_system.compute_tfidf()
run_tests(ir_system, part=2)

Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 4. - Calcul de la similarité cosinus.

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours en considérant les éléments suivants :
- Ne considérer que la mesure TF pour calculer le poids des termes de la requête (on utilise pas IDF).
- utiliser le logarithme en base 10.

Exercice : modifier la fonction `rank_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [21]:
# Exercice 4. Similarité cosinus
def rank_retrieve(self, query):
    """
    A partir d'une requête (une liste de mots), retourne une liste de documents (classés par docID) et de scores pour la requête en appliquant la simalirité cosinus.

    Dans l'exemple ci-dessous. C'est la mesure de Jaccard qui est utilisée.
    """

    tfq=defaultdict(Counter)
    itfq=defaultdict(Counter)

    tfd=defaultdict(Counter)
    itfd=defaultdict(Counter)

    for word in query :
      tfq[word] = query.count(word)
    for word in tfq :
      itfq[word] = 1+math.log10(tfq[word])

    i=0
    for doc in self.docs:
      for word in query :
        tfd[i][word] = doc.count(word)
      i+=1

    for word in query:
      i=0
      for doc in self.docs:
        if word in doc :
          i+=1
      tfd[word]=i
      itfd[word]=1+math.log10(tfd[word])

    numerateur=0
    for word in query:
      numerateur +=itfd[word]*itfq[word]

    denominateur=1
    q=d=0
    for word in query:
      q+=math.pow(itfq[word],2)
      d+=math.pow(itfd[word],2)
    denominateur=math.sqrt(q)*math.sqrt(d)

    cos=numerateur/denominateur


    scores = [0.0 for _ in range(len(self.docs))]
    query_set = set(query)
    for d in range(len(self.docs)):
        doc_set = set(self.docs[d])
        intersection = len(query_set & doc_set)
        union = len(query_set | doc_set)
        # Calcul de la similarité Jaccard
        scores[d] = intersection / union if union != 0 else 0.0
        "score = cos"

    # Tri des scores
    ranking = [idx for idx, sim in sorted(enumerate(scores),
                                        key=lambda pair: pair[1], reverse=True)]
    results = []
    for i in range(10):
        results.append((ranking[i], scores[ranking[i]]))
    return results

# Ne pas modifier les lignes ci-dessous
IRSystem.rank_retrieve = rank_retrieve
run_tests(ir_system, part=3)

===== Running tests =====
Cosine Similarity Test
    Score: 0 Feedback: 0/5 Correct. Accuracy: 0.000000
